In [45]:
from google.colab import files

# Esto abrirá el cuadro de diálogo para que selecciones tus archivos
uploaded = files.upload()

# Los archivos se subirán y estarán disponibles en el directorio actual del Colab

Saving DESeq2_results_root_drought_vs_control_SIGNIFICANT.csv to DESeq2_results_root_drought_vs_control_SIGNIFICANT.csv


In [46]:
from google.colab import files

# Esto abrirá el cuadro de diálogo para que selecciones tus archivos
uploaded = files.upload()

# Los archivos se subirán y estarán disponibles en el directorio actual del Colab

Saving E-ENAD-3-query-results.tsv to E-ENAD-3-query-results.tsv


In [47]:
import pandas as pd
import numpy as np

# --- 1. Definición de Archivos ---
# Usamos el nuevo archivo TSV (E-ENAD-3) como el set de datos completo
FILE_PRUPE_FULL = 'E-ENAD-3-query-results.tsv'
FILE_R = 'DESeq2_results_root_drought_vs_control_SIGNIFICANT.csv'
OUTPUT_FILE = 'comparacion_PRUPE_vs_PRUPE_resultados.csv'

print("Cargando y limpiando datos para la comparación PRUPE...")

# --- A. Cargar y procesar el archivo de R (PRUPE) ---
# Solución al KeyError y NA: Leer y limpiar las filas vacías iniciales
df_sig = pd.read_csv(FILE_R, header=0)
df_sig['gene'] = df_sig['gene'].astype(str).str.strip() # Limpieza de espacios en ID
df_sig_cleaned = df_sig.dropna(subset=['gene', 'log2FoldChange'])

# Preparamos las columnas para el merge
df_sig = df_sig_cleaned.set_index('gene')
df_sig = df_sig[['log2FoldChange', 'padj', 'threshold']].rename(columns={
    'log2FoldChange': 'lfc_R',
    'padj': 'padj_R',
    'threshold': 'Direction_R'
})

# --- B. Cargar y procesar el nuevo archivo completo (PRUPE/TSV) ---
# Ajuste de lectura: TSV, saltando 3 filas de encabezado de texto
df_new_full = pd.read_csv(FILE_PRUPE_FULL, sep='\t', skiprows=3, header=0)

# Renombrar columnas largas y complejas del archivo TSV para facilitar el manejo
df_new_full = df_new_full.rename(columns={
    'Gene ID': 'gene_id',
    "'drought environment' vs 'none' in 'root' .foldChange": 'lfc_atlas',
    "'drought environment' vs 'none' in 'root'.pValue": 'padj_atlas'
})

# Limpieza y selección de columnas para el merge
df_new_full['gene_id'] = df_new_full['gene_id'].astype(str).str.strip()
df_new_full.set_index('gene_id', inplace=True)
df_new_full = df_new_full[['lfc_atlas', 'padj_atlas']]

# --- 2. Unión de los DataFrames (Inner Join) ---
# Une los dos sets de datos usando el ID de gen PRUPE
df_comparison = df_sig.merge(
    df_new_full,
    left_index=True,
    right_index=True,
    how='inner'
)
print(f"Coincidencias encontradas: {len(df_comparison)} genes comunes.")

# --- 3. Análisis de Magnitud, Dirección y Significancia ---

# Magnitud: Calcular la diferencia absoluta en Log2 Fold Change
df_comparison['lfc_difference'] = np.abs(
    df_comparison['lfc_R'].astype(float) - df_comparison['lfc_atlas'].astype(float)
)

# Dirección: Determinar si la dirección (Up/Down) coincide
df_comparison['Direction_atlas'] = np.where(df_comparison['lfc_atlas'].astype(float) > 0, 'Up', 'Down')
df_comparison['Direction_match'] = (df_comparison['Direction_atlas'] == df_comparison['Direction_R'])

# Selección y ordenamiento final
df_final = df_comparison[[
    'lfc_R', 'lfc_atlas', 'lfc_difference',
    'padj_R', 'padj_atlas',
    'Direction_R', 'Direction_atlas', 'Direction_match'
]].sort_values(by='lfc_difference', ascending=False)

# --- 4. Guardar los resultados ---
df_final.to_csv(OUTPUT_FILE)
print(f"Análisis completado. Archivo de resultados guardado como: {OUTPUT_FILE}")

Cargando y limpiando datos para la comparación PRUPE...
Coincidencias encontradas: 805 genes comunes.
Análisis completado. Archivo de resultados guardado como: comparacion_PRUPE_vs_PRUPE_resultados.csv


In [48]:
# Descargar el archivo de resultados a tu computadora
try:
    files.download(OUTPUT_FILE)
    print(f"\n¡Éxito! El archivo {OUTPUT_FILE} ha sido descargado.")
except:
    print("\nError al intentar descargar el archivo. Verifícalo en la carpeta de Colab.")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


¡Éxito! El archivo comparacion_PRUPE_vs_PRUPE_resultados.csv ha sido descargado.
